# Fixed-Point Quantisation of CNN

This tutorial introduces fixed-point quantisation and the effect that it can have on CNNs.

This tutorial is organised as follows:

1. [Fixed point or Q representation](#Fixed-point-or-Q-representation)
2. [Conversions](#Conversions)
3. [Arithmetic Operations](#Arithmetic-operations)
4. [CNNs and Fixed Point Representation](#CNNs-and-Fixed-Point-Representation)
5. [Exercise](#Exercise)

### Intended Outcome:
1. Learn about Fixed-point/Q Representation
2. Perform mathematical operations with Fixed-point representation
3. Understand the effect it can have on real-life applications e.g.: speed-up or sometimes lower accuracy (!)

## Fixed point or Q representation

As a quick recapitulation: to represent a non-integer or a fractional number a developer usually has two options. The first one is to use floating point representation, which supports a trade-off between numerical range and precision and resolution respectively. The name gives out the main property, the point separating the integer part and fractional point is floating, rather than staying fixed. However, this data-type and its arithmetic is challenging to implement in hardware if we want to achieve optimal performance, unless the processing device has enough space and resources for a dedicated *F*loating *P*oint *U*nit (FPU).

That is why in most of low-power, low-performance embeded devices, that might require constant resolution, we find fixed point representation or *Q*-representation as we have talked before. These bits are then split into two parts, with an imaginary decimal point separating them. The first part is for the _Integer_ part (IP) and the second one is _Fractional_ part. For example, given that we are operating on 16 bits in total, a Q16 number has 16 fractional bits; a Q2.14 number has 2 integer bits and 14 fractional bits. Note, that to represent signed numbers, we usually need to assign one more bit from the integer part to determine the number being signed.

This representation has its pros and its cons, on one hand it is very easy to [implement](goo.gl/oka37Z) it in low-level designs, giving improved performance and lower power consumption, on the other hand the issue remains its precision. Let's see that on an example.

In [10]:
import numpy as np

#Fractional bits
f = 2

a = np.linspace(1,2,10)

#Scale the output as if was represented with only 2 fractional bits
a_fix = np.round(a*f)*(1.0/f)

print(a)
print(a_fix)

[1.         1.11111111 1.22222222 1.33333333 1.44444444 1.55555556
 1.66666667 1.77777778 1.88888889 2.        ]
[1.  1.  1.  1.5 1.5 1.5 1.5 2.  2.  2. ]


### Two's Complement Fixed Point Representation
Throughout this tutorial we are going to be talking mainly about dynamic fixed point representation, another option as presented in the lecture series is Two's Complement Fixed Point Representation. The difference between this representation and the two's complement one is that the MSB represents basically a negative number. E.g.: you would like to represent -3 with three bits, in two's complement you would simply write 101 in our representation 1 ← Signed and then 11 or 111 in total. Simply put the number is always "positively" represented with the first bit denoting sign. 

## Conversions
##### Float to Q
To convert the number from floating point to Qm.n format: 
1. Multiply the floating point number by 2<sup>n</sup> - which is basically a shift of a number left by *n* places
2. Round to the nearest integer

##### Q to float
1. Convert the number to floating point as if it was an integer, in other words remove the binary point
2. Multiply by 2<sup>-n</sup>

In [11]:
# Given that we have UQ8.8 format (UQm.n) 
m = 8
n = 8
f = 2.1
q = f * 2**n
rounded = round(q)
print("The number in Q format is: {} and its binary representation is: {} and hex is: {}".format(rounded, bin(rounded), hex(rounded)))
print("The number back as a float is: {}".format(rounded*2**(-n)))

The number in Q format is: 538 and its binary representation is: 0b1000011010 and hex is: 0x21a
The number back as a float is: 2.1015625


This is another problem that occurs with this representation, if you are performing conversion from one format to another, you are loosing precision and eventually, accuracy. Keep in mind that when you are performing arithmetic operations, you have to saturate the output to fit into the bounds of sensible representation.

## Arithmetic operations
One big advantage of fixed point representation is that, arithmetic functions can be performed directly on an ALU. Given that we have UQ8.8 representation we can demonstrate it with couple of examples.

##### Addition & Subtraction

In [12]:
i = 8
f = 8
a = 2.1
b = 2.3
a_fp = round(a * 2**f)
b_fp = round(b * 2**f)
c_fp = a_fp + b_fp
d_fp = a_fp - b_fp
print("The addition back as a float is: {}, using floats: {}".format(c_fp*2**(-f),a+b))
print("The subtraction back as a float is: {}, using floats: {}".format(d_fp*2**(-f),a-b))

The addition back as a float is: 4.40234375, using floats: 4.4
The subtraction back as a float is: -0.19921875, using floats: -0.19999999999999973


##### Multiplication
Multiplication is a little bit complicated and we have to introduce the concept of saturation. In case of overflow (which can also happen in case of addition and subtraction) we have to keep in mind to clamp the result between sensible values of the range of our representation.

In [13]:
def saturate(x):
    if x>0xFFFF:
        return 0xFFFF
    elif x<0x0:
        return 0x0
    else:
        return x

i = 8
f = 8
K = 1 << (f-1)

a = 2.1
b = 2.3

a_fp = round(a * 2**f)
b_fp = round(b * 2**f)

temp = a_fp * b_fp
temp += K
result = saturate(temp >> f)
print("The multiplication back as a float is: {}, using floats: {}".format(result*2**(-f),a*b))

The multiplication back as a float is: 4.8359375, using floats: 4.83


#### Division
Again using the same concepts: 

In [14]:
i = 8
f = 8

a = 2.1
b = 2.3

a_fp = round(a * 2**f)
b_fp = round(b * 2**f)

# Pre-multiply by the base (Upscale to Q24 so that the result will be in Q16 format) 
temp = a_fp << f;

# Rounding: mid values are rounded up (down for negative values)
if((temp >= 0 and b_fp >= 0) or (temp < 0 and b_fp < 0)):   
    temp += b_fp / 2;  
else: 
    temp -= b_fp / 2;    

result = temp / b_fp;
print("The division back as a float is: {}, using floats: {}".format(result*2**(-f),a/b))

The division back as a float is: 0.9153656886672326, using floats: 0.9130434782608696


One might be asking how about if we have a combination of different Q representations. Then, we have to somehow concert the values and subsequently round them, but how? 

The conversion often involves a right shift operation to eliminate bits from higher-order Q representation e.g. Q16 to Q8. Before throwing these bits away we should consider performing a rounding operation. This reduces bias and can improve the accuracy of the overall result. A common technique rather than completely throw away bits is to add one to the MSB of fractional part e.g.:

In [15]:
#Q4.4
i = 4
f = 4
a = 1.1
a_fp = round(a * 2**f)
print("Hexadecimal fixed point representation: {}".format(hex(a_fp)))
print("Fixed point representation, as float: {}".format(a_fp*2**(-f)))

# Converting into Q4.2
# Rounding by adding one to the MSB
f = 2
t_fp = a_fp
a_fp+=0x02
a_fp>>=2
t_fp>>=2
print("Converted fixed point representation, as float: {}".format(a_fp*2**(-f)))
print("Truncated fixed point representation, as float: {}".format(t_fp*2**(-f)))

Hexadecimal fixed point representation: 0x12
Fixed point representation, as float: 1.125
Converted fixed point representation, as float: 1.25
Truncated fixed point representation, as float: 1.0


Another popular method is to round the number to the nearest integer that can potentially represent this number, once converted into an integer representation. This method gives the closest representation of our original number 1.1 .

In [16]:
from math import floor

i = 4
f = 4
a = 1.1
scale = 1 << f
rounded_and_scaled = floor(a * scale + 0.5)
print("Round-to-Nearest, as a float: {}".format(rounded_and_scaled*2**(-f)))

Round-to-Nearest, as a float: 1.125


More advanced techniques usually used round differently based on the bit number/position and whether it is odd or even etc. More can be found at this [link](http://en.wikipedia.org/wiki/Rounding). 

## CNNs and Fixed Point Representation

These techniques, although they might seem to be very imprecise, have been proven to be very efficient especially in relation with machine learning, where we have many intermediate results, weights, biases, where computation could be potentially accelerated ([link](https://arxiv.org/abs/1712.05877)) by lower data-bandwidth. This is especially true if we consider embedded systems and FPGAs, which might not have sufficient computing and memory capabilities to support floats. 

One has to be careful to chose the correct representation and Q format for the desired application and tradeoff between accuracy, resources and speed. Let us look at an example.
In the previous tutorials 1 & 2 we read about the face detection task and a SSD architecture which was used to solve this problem by using floats, but we can replace them and use fixed point representation. 

For those who do not know about SSD (Single Shot MultiBox Detector) by C. Szegedy et al. it was released at the end of November 2016 and reached new records in terms of performance and precision for object detection tasks, scoring over 74% mAP (mean Average Precision) at 59 frames per second on standard datasets such as PascalVOC and COCO. It is called *Single Shot*, because the task of localization and classification are done in a single forward pass of the network and *MultiBox* is the name of a technique four bounding box regression developed by Szegedy et al.. This network is exciting because it is promising especially in terms of embedded systems because of its very low overhead. You can find out more at: [Paper](https://arxiv.org/abs/1512.02325) or as a [Blog](https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-495ef744fab) post. 

In [17]:
from IPython.display import HTML, display
display(HTML("<table><tr><td>Original input data<img src='../data/figs/original.jpg'></td><td>SQ15.16<img src='../data/figs/fixed_32_16Q16.jpg'></td><td>SQ7.8<img src='../data/figs/fixed_16_8Q8.jpg'></td><td>SQ29.2<img src='../data/figs/fixed_32_30Q2.jpg'></td><td>SQ1.15<img src='../data/figs/fixed_16_2Q14.jpg'></td></tr></table>"))

Original input data,SQ15.16,SQ7.8,SQ29.2,SQ1.15


These are the results in the SSD example by using SQ15.16 and SQ7.8, the differences are not that significant. However, if using different proportions e.g.: Q30.2 and Q14.2 the results are distorted.

This can be seen in greater detail at a MNIST classification challange using LeNet CNN architecture. The settings for the fractional part and integer part can be very delicate and you can see that the accuracy can be very sensitive about this. Note that in each layer, weights inputs and outputs use the same fixed point scheme.

![Flowchart.png](../data/figs/accuracy_vs_fb.png)

## Exercise

Let's look at the range of the numbers and it's resolution e.g.: in *U* (Unsigned) Q2.14 and *S* (Signed, aka one bit from the integer part will represent the sign) Q8.8, how would you calculate it?

In [18]:
# Please replace the zeros with reasonable values
# In total we have 16 bits to operate on
# UQ2.14
i = 2
f = 14
print("UQ2.14")
print("Range is: {} to {}".format(0,0))
print("Resolution is: {}".format(0))

# SQ8.8
i = 8
f = 8
print("SQ8.8")
print("Range is: {} to {}".format(0,0))
print("Resolution is: {}".format(0))

UQ2.14
Range is: 0 to 0
Resolution is: 0
SQ8.8
Range is: 0 to 0
Resolution is: 0
